# Configure Access Control List using Yang Model
<sup>Authored by Annapoorani G (annag@cisco.com)</sup>


Typically, CLIs are widely used for configuring and extracting the operational details of a router. But the general mechanism of CLI scraping is not flexible and optimal. Small changes in the configuration require rewriting scripts multiple times. Bulk configuration changes through CLIs are cumbersome and error-prone. These limitations restrict automation and scale. To overcome these limitations, you need an automated mechanism to manage your network.

Cisco IOS XR supports a programmatic way of configuring and collecting operational data of a network device using data models. They replace the process of manual configuration, which is proprietary, and highly text-based. The data models are written in an industry-defined language and is used to automate configuration task and retreive operational data across heterogeneous devices in a network.
Model-driven programmability provides a simple, flexible and rich framework for device programmability. This programmability framework provides multiple choices to interface with an IOS XR device in terms of transport, protocol and encoding. These choices are decoupled from the models for greater flexibility.

>In this notebook, let's explore how to configure ACL on a router using Yang model.


# Configuration Steps


* [Bring Up Emulator](#step1)
* [Configure ACL using Yang](#step2)
* [Verify the ACL configuration](#step3)




### <a name="step1"></a>Bring up Your Emulator

>First, you'll have to set up the emulator to build your network. The code below helps you set up the IOS XR configurations for a 2 router topology for our virtual test bed and a Linux server to use on the same LAN, with SSH to all of the routers.
To run the code, select the cell  cell and then press the 'play' button on the bar above. 


In [1]:
from acl import *
sim = Vxr()
sim.no_image_copy=True
sim.clean()
print("Sim clean: Done")
print("Simulation starting. Please wait for the Sim status message. This may take 3-10 minutes.")

try:
    sim.start(cfg)
    ports=sim.ports()
    print(ports)
    status = sim.status()
    print("Sim status: ", status)
except Exception as err:
    print("Sim launch failed (%s)" % str(err))

INFO:pyvxr.vxr:v1.1.0 2021-04-23 05:58 output_dir:vxr.out
INFO:pyvxr.vxr:b10e8f6b8c14:/home/vxr/notebooks/Put-Technology-to-Work/ACL
INFO:pyvxr.vxr:Extracting vxr version from '/opt/cisco/vxr2/latest/setup.sh' file.
INFO:pyvxr.vxr_session:Starting a local bash session for user:vxr


3.8.8 (default, Apr 13 2021, 19:58:26) 
[GCC 7.3.0]
Sim clean: Done
Simulation starting. Please wait for the Sim status message. This may take 3-10 minutes.


INFO:pyvxr.sim:Launch: sim_dir:/nobackup/vxr/pyvxr/lf1mp6px4n sim_rel:/opt/cisco/vxr2/latest
INFO:pyvxr.sim:Stopping previous simulation (if any)
INFO:pyvxr.sim:Cleaning previous simulation (if any)
INFO:pyvxr.sim:Starting vxr: 'sim --skiphomecheck -n '
INFO:pyvxr.sim:Vxr up on host localhost
INFO:pyvxr.vxr:Getting port vector files for:r1, r2
INFO:pyvxr.console:r2:wait for XR login prompt (console output captured in vxr.out/logs/console.r2.log)
INFO:pyvxr.console:r1:wait for XR login prompt (console output captured in vxr.out/logs/console.r1.log)
INFO:pyvxr.console:r1:entering new XR username 'cisco', password 'cisco123'
INFO:pyvxr.console:r1:entering XR username 'cisco', password 'cisco123'
INFO:pyvxr.console:r2:entering new XR username 'cisco', password 'cisco123'
INFO:pyvxr.bringup:r1:login successful
INFO:pyvxr.bringup:r1:wait for IOS XR RUN state
INFO:pyvxr.bringup:r1:run state RPs:1 (expected:1) LCs:0 (expected:0)
INFO:pyvxr.bringup:r1:all nodes reached IOS XR RUN state
INFO:pyv

{'r1': {'HostAgent': '172.17.0.2', 'dual_rp': False, 'linecard_slots': [0], 'monitor0': 43323, 'plugin': '8201', 'redir0': 0, 'serial0': 43382, 'serial1': 37609, 'xr_mgmt_ip': '192.168.254.155', 'xr_redir22': 61928, 'xr_redir23': 64763, 'xr_redir830': 64725}, 'r2': {'HostAgent': '172.17.0.2', 'dual_rp': False, 'linecard_slots': [0], 'monitor0': 42951, 'plugin': '8201', 'redir0': 0, 'serial0': 38622, 'serial1': 33989, 'xr_mgmt_ip': '192.168.254.156', 'xr_redir22': 60414, 'xr_redir23': 62640, 'xr_redir830': 64424}}
Sim status:  {'localhost': 'running'}


>At this point, console access to the router is available. Optionally, you can access the simulated router consoles directly from your laptop  through ssh.
<b>Note the host and port niumber from the output. You muct update the the code when applying the yang model.<b>

In [2]:
print('Consoles can be reached by:')
print(get_telnet_cmd(sim, 'r1'), '\n', get_telnet_cmd(sim, 'r2'))
print('or better:')
print(get_ssh_cmd(sim, 'r1'), '\n', get_ssh_cmd(sim, 'r2'))
print('The password is cisco123')

Consoles can be reached by:
telnet 172.17.0.2 43382 
 telnet 172.17.0.2 38622
or better:
ssh cisco@172.17.0.2 -p61928 
 ssh cisco@172.17.0.2 -p60414
The password is cisco123


In [3]:
ports = sim.ports()
r1_host = str(ports['r1']['HostAgent'])
r1_port = str(ports['r1']['xr_redir22'])
print(r1_host, r1_port)

172.17.0.2 61928


### <a name="step2"></a>Configure ACL using Yang

>Prerequistes:  You've to install ncclient to run your yang model.

In [4]:
pip install ncclient

Note: you may need to restart the kernel to use updated packages.


>Configure ACL using the yang.xml model. You can update the permit and deny statements and also update the interface where you want to apply ACL. Ensure to configure passwordless shh connection. See https://help.dreamhost.com/hc/en-us/articles/216499537-How-to-configure-passwordless-login-in-Mac-OS-X-and-Linux.

In [5]:
from ncclient import manager
from ncclient.xml_ import to_ele
from lxml import etree

aclconfig = '''
<config>
<ipv4-acl-and-prefix-list xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ipv4-acl-cfg">
   <accesses>
    <access>
     <access-list-name>myacl</access-list-name>
     <access-list-entries>
      <access-list-entry>
       <sequence-number>2</sequence-number>
       <grant>permit</grant>
       <source-network>
        <source-address>10.5.7.1</source-address>
       </source-network>
       <sequence-str>2</sequence-str>
      </access-list-entry>
      <access-list-entry>
       <sequence-number>5</sequence-number>
       <grant>deny</grant>
       <source-network>
        <source-address>198.51.100.1</source-address>
       </source-network>
       <sequence-str>5</sequence-str>
      </access-list-entry>
      <access-list-entry>
       <sequence-number>10</sequence-number>
       <grant>deny</grant>
       <protocol>tcp</protocol>
       <sequence-str>10</sequence-str>
      </access-list-entry>
     </access-list-entries>
    </access>
   </accesses>
  </ipv4-acl-and-prefix-list>
 <acl xmlns="http://openconfig.net/yang/acl">
   <interfaces>
    <interface>
     <id>FourHundredGigE0/0/0/1</id>
     <config>
      <id>FourHundredGigE0/0/0/1</id>
     </config>
     <interface-ref>
      <config>
       <interface>FourHundredGigE0/0/0/1</interface>
      </config>
     </interface-ref>
     <ingress-acl-sets>
      <ingress-acl-set>
       <set-name>myacl</set-name>
       <type>ACL_IPV4</type>
      </ingress-acl-set>
     </ingress-acl-sets>
    </interface>
   </interfaces>
 </acl>
</config>
'''


                
def connect(host, port, user, password, source):
      conn = manager.connect(host=host,
                             port=port,
                             username=user,
                             password=password,
                             device_params={'name': 'iosxr'},
                             hostkey_verify=False,
                             allow_agent=False,
                             look_for_keys=False                         
                            )

      rpc_reply = conn.edit_config(config=aclconfig)
      conn.commit()
      print(rpc_reply)

connect(r1_host, r1_port, 'cisco', 'cisco123', 'candidate')




INFO:ncclient.transport.ssh:Connected (version 2.0, client Cisco-2.0)
INFO:ncclient.transport.ssh:Authentication (password) successful!
INFO:ncclient.transport.ssh:[host 172.17.0.2 session 0x7f3dc00a5e50] Sending:
b'<?xml version="1.0" encoding="UTF-8"?><nc:hello xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0"><nc:capabilities><nc:capability>urn:ietf:params:netconf:base:1.0</nc:capability><nc:capability>urn:ietf:params:netconf:base:1.1</nc:capability><nc:capability>urn:ietf:params:netconf:capability:writable-running:1.0</nc:capability><nc:capability>urn:ietf:params:netconf:capability:candidate:1.0</nc:capability><nc:capability>urn:ietf:params:netconf:capability:confirmed-commit:1.0</nc:capability><nc:capability>urn:ietf:params:netconf:capability:rollback-on-error:1.0</nc:capability><nc:capability>urn:ietf:params:netconf:capability:startup:1.0</nc:capability><nc:capability>urn:ietf:params:netconf:capability:url:1.0?scheme=http,ftp,file,https,sftp</nc:capability><nc:capability>urn:iet

<?xml version="1.0"?>
<rpc-reply message-id="urn:uuid:d2bea575-20ad-4c06-b905-f004ee8606af" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
 <ok/>
</rpc-reply>



> (optional) Retrieve the ACL yang model. 

### <a name="step3"></a>Verify the ACL configuration

>Access the telnet consoles of the router to verify the telemetry configuration.

In [6]:
import telnetlib
ports = sim.ports()
loginr1 = telnetlib.Telnet(str(ports['r1']['HostAgent']) , str(ports['r1']['serial0']))
loginr1.write(('''
show ip access-lists
''').encode('ascii'))
line = loginr1.read_until(b'/r/n',2)
print("***** VIEW FROM TELNET CONSOLE *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE *****

RP/0/RP0/CPU0:P1#show ip access-lists
Fri Apr 23 06:02:54.769 UTC
ipv4 access-list myacl
 2 permit ipv4 host 10.5.7.1 any
 5 deny ipv4 host 198.51.100.1 any
 10 deny tcp any any
RP/0/RP0/CPU0:P1#


>Once you are done with experimenting on the topology, you should bring down the emulator by executing the following steps:

In [7]:
# close our telnet
loginr1.close()

In [8]:
# Close the sim
sim.stop()
sim.clean()
# Clean up sim scratch space
shutil.rmtree(sim_dir)

INFO:pyvxr.vxr:Stopping sim on host localhost (dir /nobackup/vxr/pyvxr/lf1mp6px4n)
INFO:pyvxr.sim:Stopping previous simulation (if any)
INFO:pyvxr.vxr:Cleaning sim on host localhost (dir /nobackup/vxr/pyvxr/lf1mp6px4n)
INFO:pyvxr.sim:Stopping previous simulation (if any)
INFO:pyvxr.sim:Cleaning previous simulation (if any)
